# TP - Computação Natural
#### "Predict whether a mammogram mass is benign or malignant"

1. BI-RADS assessment: 1 to 5 (ordinal)  
2. Age: patient's age in years (integer)
3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
6. Severity: benign=0 or malignant=1 (binominal)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from scipy import stats
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, RMSprop, Adam

## Get the Data

In [ ]:
data = pd.read_csv('mammographic_masses.data.txt')
data

** Convert missing data (indicated by a ?) into NaN and add the appropriate column names (BI_RADS, age, shape, margin, density, and severity) **

In [ ]:
data = data.replace('?',np.nan)
data.columns = ['BI_RADS','Age','Shape','Margin','Density','Severity']
data

** Drop BI_RADS column because it has no influence on the severity forecast **

In [ ]:
data = data.drop(columns=['BI_RADS'])

** Convert datatype 'object' to 'float64' **  

In [ ]:
data.info()

In [ ]:
data = data.astype(float)
data

In [ ]:
data.info()

In [ ]:
data.describe()

### Analysing missing values
**First we get the missing values per feature.**

*Lets check them out as well*

In [ ]:
missing_values_feature = data.isnull().sum(axis=0)
graph = missing_values_feature.drop(labels='Severity')
graph

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.bar(graph.axes[0].to_list(), graph.values)

*We then develop a heatmap to give us some more information*

In [ ]:
sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

*Finally we can check the percentage of missing values per feature*

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
print(missing_value_df)

**After analysing the columns, we should have a look at the rows**

In [ ]:
data_missing = len(data.columns) - (data.apply(lambda x: x.count(), axis=1))
missing_values_data_rows = pd.DataFrame({'data_missing':data_missing})
missing_values_data_rows.sort_values('data_missing',inplace=True,ascending=False)
missing_values_data_rows

**Now lets analyse the missing data per class (Severity = 0 or Severity = 1).**

*First we group the missing values per class*

In [ ]:
grouped_data = data.groupby('Severity')
missing_values_class = grouped_data.count().rsub(grouped_data.size(), axis=0)
missing_values_class

*Now we split the dataframe per class so we can draw our plot*

In [ ]:
m_new_1, m_new_2 = missing_values_class.head(1), missing_values_class.tail(1)

In [ ]:
x = np.arange(len(m_new_1.axes[1].to_list()))
width = 0.4

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, m_new_1.values[0], width=width, label = "Severity 0")
rects2 = ax.bar(x + width/2, m_new_2.values[0], width=width, label = "Severity 1")

ax.set_xticks(x)
ax.set_xticklabels(m_new_1.axes[1].to_list())
ax.legend()

**Finally, for each class we're going to calculate the number of rows that have 1 and 2 NaN values**

In [ ]:
rows_mv1_sv0 = 0
rows_mv2_sv0 = 0
rows_mv1_sv1 = 0
rows_mv2_sv1 = 0

In [ ]:
for index, row in data.iterrows():
    if(row['Severity'] == 0):
        if(row.isnull().sum() == 1):
            rows_mv1_sv0 += 1
        elif(row.isnull().sum() == 2):
            rows_mv2_sv0 += 1
    else:
        if(row.isnull().sum() == 1):
            rows_mv1_sv1 += 1
        elif(row.isnull().sum() == 2):
            rows_mv2_sv1 += 1

*We create a dataframe only for visualization purpose*

In [ ]:
numberofnan_class = pd.DataFrame(np.array([[rows_mv1_sv0,rows_mv2_sv0], [rows_mv1_sv1,rows_mv2_sv1]]), 
                                    index=['Severity 0','Severity 1'], columns=['1 NaN', '2 NaN'])
numberofnan_class

In [ ]:
labels = ['1 NaN', '2 NaN']
x = np.arange(len(labels))
width = 0.4

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, [rows_mv1_sv0,rows_mv2_sv0], width=width, label = "Severity 0")
rects2 = ax.bar(x + width/2, [rows_mv1_sv1,rows_mv2_sv1], width=width, label = "Severity 1")

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

*With this information we can also see the number of rows with 1 or 2 missing values per class*

In [ ]:
nan_class = pd.DataFrame(np.array([[rows_mv1_sv0+rows_mv2_sv0], [rows_mv1_sv1+rows_mv2_sv1]]), 
                                    index=['Severity 0','Severity 1'], columns=['Sum'])
nan_class

In [ ]:
plt.figure(figsize=(3, 5))
plt.bar(['Severity 0','Severity 1'],[rows_mv1_sv0+rows_mv2_sv0,rows_mv1_sv1+rows_mv2_sv1])

** The missing data seems randomly distributed, so we decided to go with the following strategy: **

* Drop rows with 2 NaN values

* Replace the NaN values from rows with 1 missing value

*First we get the mode of every feature for each class*

In [ ]:
mode_sv0 = data[data['Severity'] == 0].mode()
mode_sv1 = data[data['Severity'] == 1].mode()
mode_sv0 = mode_sv0.drop([1])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(mode_sv0)
    print(mode_sv1)

*After we create conditions to replace the NaN values on rows with 1 missing value.*

*For that we need the index of the rows which have 1 missing value.*

In [ ]:
rows_1nan = missing_values_data_rows.index[missing_values_data_rows['data_missing'] == 1].tolist()
mask_sv0 = (data['Severity'] == 0) & (data.index.isin(rows_1nan))
mask_sv1 = (data['Severity'] == 1) & (data.index.isin(rows_1nan))

*We can now proceed and replace the missing values for their class mode* 

In [ ]:
data.loc[mask_sv0, 'Shape'] = data.loc[mask_sv0, 'Shape'].fillna(mode_sv0.loc[0,'Shape'])
data.loc[mask_sv0, 'Margin'] = data.loc[mask_sv0, 'Margin'].fillna(mode_sv0.loc[0,'Margin'])
data.loc[mask_sv0, 'Density'] = data.loc[mask_sv0, 'Density'].fillna(mode_sv0.loc[0,'Density'])
data.loc[mask_sv1, 'Age'] = data.loc[mask_sv1, 'Age'].fillna(mode_sv1.loc[0,'Age'])
data.loc[mask_sv1, 'Shape'] = data.loc[mask_sv1, 'Shape'].fillna(mode_sv1.loc[0,'Shape'])
data.loc[mask_sv1, 'Margin'] = data.loc[mask_sv1, 'Margin'].fillna(mode_sv1.loc[0,'Margin'])
data.loc[mask_sv1, 'Density'] = data.loc[mask_sv1, 'Density'].fillna(mode_sv1.loc[0,'Density'])
data

*Finally, we can drop rows with NaN values because the only ones that are left are the ones with 2 NaN*

In [ ]:
data = data.dropna()
data.index = np.arange(1, len(data) + 1)
data

In [ ]:
data.describe()

### Detect Outliers using Box plot (Uni-variate outlier)

### Detect outliers using mathematical function Z-Score

In [ ]:
z = np.abs(stats.zscore(data))
threshold = 3
print(np.where(z > threshold))
# The first array contains the list of row numbers and second array respective column numbers

Column 3 (density) has all outliers

### Detect outliers using IQR Score
Similar to Z-Score

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
iqr = Q3 - Q1
print(iqr)

In [ ]:
# Não curti ...
#print(data < (Q1 - 1.5 * iqr)) |(data > (Q3 + 1.5 * iqr))

## Data Preparation

### Remove Outliers using Z-Score

##### + explanations: https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

In [ ]:
# Só fazer 1 vez
data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
data.index = np.arange(1, len(data) + 1)
data

### Converting pandas dataframes to numpy arrays

In [ ]:
#X = data.drop('Severity',axis=1).to_numpy()
#y = data['Severity'].to_numpy()
#X

### Normalizing the attribute data using StandardScaler

** Fit scaler to the features **

In [ ]:
# Só fazer 1 vez
#scaler = StandardScaler()
#scaler.fit(X)
#scaler.mean_

** Transform the features to a scaled version **

In [ ]:
#X = scaler.transform(X)
#X

In [ ]:
col_names = ['Age']
features = data[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
data[col_names] = features

one_hot = pd.get_dummies(data['Shape'])
data = data.drop('Shape',axis = 1)
data = data.join(one_hot)
data = data.rename(columns={1.0: "Shape_1", 2.0: "Shape_2", 3.0: "Shape_3", 4.0: "Shape_4"})

one_hot = pd.get_dummies(data['Margin'])
data = data.drop('Margin',axis = 1)
data = data.join(one_hot)
data = data.rename(columns={1.0: "Margin_1", 2.0: "Margin_2", 3.0: "Margin_3", 4.0: "Margin_4", 5.0: "Margin_5"})

one_hot = pd.get_dummies(data['Density'])
data = data.drop('Density',axis = 1)
data = data.join(one_hot)
data = data.rename(columns={1.0: "Density_1", 2.0: "Density_2", 3.0: "Density_3", 4.0: "Density_4"})

one_hot = pd.get_dummies(data['Severity'])
data = data.drop('Severity',axis = 1)
data = data.join(one_hot)
data = data.rename(columns={0.0: "Severity_0", 1.0: "Severity_1"})


data

In [ ]:
X = data.drop(['Severity_0','Severity_1'],axis=1).to_numpy()
y = data[['Severity_0','Severity_1']].to_numpy()

In [ ]:
np.info(X)
print("---")
np.info(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
np.info(X_train)
print("---")
np.info(X_test)
print("---")
np.info(y_train)
print("---")
np.info(y_test)

## Neural Networks

In [ ]:
def buildModel(hidden_layers, nodes_per_layer, activation_fn, optimizer, loss_fn, metrics, inputs=13,):
    model = Sequential()
    #add input layer
    model.add(Dense(inputs, activation=activation_fn, input_shape=(inputs,)))

    #add hidden layers    
    for i in range(hidden_layers):
        model.add(Dense(nodes_per_layer, activation=activation_fn))

    #add output layer
    model.add(Dense(2,activation=activation_fn))
    
    if(optimizer=='rmsprop'): optimizer = RMSprop(learning_rate=0.1)

    #compile model
    model.compile(optimizer, loss=loss_fn, metrics=metrics)

    return model

def predictionsModel(model, input_attributes, output_attributes):
    print("\n###########predict###########\n")
    previsoes = model.predict(input_attributes)
    
    # arredondar para 0 ou 1 pois pretende-se um output binário
    LP = []
    
    f = lambda x: int(round(x))
    vfunc = np.vectorize(f)
    
    for prev in previsoes:
        prev = vfunc(prev)
        LP.append(prev)
        
    for i in range(len(output_attributes)):
        print("Target:", output_attributes[i], " Prediction:", LP[i])
        if i > 30:
            break

In [ ]:
testModel = buildModel(hidden_layers=3, nodes_per_layer=16, activation_fn='softmax',  optimizer='rmsprop', loss_fn='binary_crossentropy', metrics=['accuracy'])


history = testModel.fit(X_train, y_train, validation_split=0.33, epochs=50, batch_size=64, verbose=2)


# Visualize history
# Plot history: Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


print("###########evaluate###########\n")
scores = testModel.evaluate(X_test, y_test, batch_size=64, verbose=1)
print("\n metrica: %s: %.2f%%\n" % (testModel.metrics_names[1], scores[1]*100))


predictions = predictionsModel(testModel, X_test, y_test)

In [ ]:
num_folds = 10
acc_per_fold = []
loss_per_fold = []
scores=[]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no=1

for train, test in kfold.split(X, y):
    testModel = buildModel(hidden_layers=3, nodes_per_layer=16, activation_fn='softmax',  optimizer='rmsprop', loss_fn='binary_crossentropy', metrics=['accuracy'])

    history = testModel.fit(X[train], y[train],
          epochs=50,
          batch_size=64,
          verbose=0)

    score = testModel.evaluate(X[test], y[test], batch_size=64, verbose=0)
    
    predictions = predictionsModel(testModel, X[test], y[test])
    
    acc_per_fold.append(score[1] * 100)
    loss_per_fold.append(score[0])
    
    print("Fold %d: loss = %.2f || accuracy=  %.2f%%" % (fold_no, score[0], score[1]*100))

    fold_no+=1

print('Average scores for all folds:')
print("> Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(acc_per_fold),np.std(acc_per_fold)))
print("> Loss: %.2f " % (np.mean(loss_per_fold)))